In [0]:
import torch

In [2]:
print(torch.cuda.torch.cuda.is_available())

True


In [3]:
torch.rand(5,2)

tensor([[0.0790, 0.2342],
        [0.3671, 0.2059],
        [0.3315, 0.1161],
        [0.1127, 0.2137],
        [0.6229, 0.9178]])

In [4]:
torch.rand(5, 3)


tensor([[0.7061, 0.5665, 0.6130],
        [0.6824, 0.3539, 0.3668],
        [0.3889, 0.1290, 0.3498],
        [0.7504, 0.5181, 0.6194],
        [0.2255, 0.9204, 0.1359]])

In [5]:
torch.tensor([5.5, 3])

tensor([5.5000, 3.0000])

In [0]:
import numpy as np

In [7]:
torch.from_numpy(np.arange(10))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
ten = torch.from_numpy(np.arange(10))

In [9]:
ten + ten

tensor([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [4]:
ten * ten

tensor([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [0]:
import torch.nn as nn
from torch.autograd import Variable

# Redes Neurais
## NN de 1 neurônio

In [0]:
class Net(nn.Module):
  
  def __init__(self):
    # Inicialization of the network
    super(Net, self).__init__()
    # One linear unit
    self.fc1 = nn.Linear(1, 1)
    
  def forward(self, X):
    # Pass it to the layer
    return self.fc1(X)
  

In [0]:
net = Net()

In [11]:
print(net)
print()
print(list(net.parameters()))

Net(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)

[Parameter containing:
tensor([[0.3462]], requires_grad=True), Parameter containing:
tensor([-0.2998], requires_grad=True)]


In [34]:
inp = Variable(torch.randn(1,1,1), requires_grad=True)
print(inp)

tensor([[[0.4179]]], requires_grad=True)


In [14]:
out = net(inp)
print(out)

tensor([[[-0.0739]]], grad_fn=<AddBackward0>)


In [17]:
# w * x + bias
0.3462 * 0.6524 - 0.2998

-0.07393912000000002

In [0]:
def loss_function(pred, expected):
  return (expected - pred)**2

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [0]:
data = [(1, 4), (2, 7), (3, 10), (4, 13), (5, 16), (6, 19)]

In [38]:
for epoch in range(100):
  for i, dat in enumerate(data):
    X, Y = iter(dat)
    X = Variable(torch.FloatTensor([X]), requires_grad=True)
    Y = Variable(torch.FloatTensor([Y]), requires_grad=False)
    optimizer.zero_grad()
    outs = net(X)
    loss = loss_function(outs, Y)
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0 and epoch % 10 == 0:
      print('Epoch {} | Loss: {}'.format(epoch, loss.data[0]))
    

Epoch 0 | Loss: 25.93330192565918
Epoch 10 | Loss: 0.4031047821044922
Epoch 20 | Loss: 0.25242292881011963
Epoch 30 | Loss: 0.15806618332862854
Epoch 40 | Loss: 0.09898065775632858
Epoch 50 | Loss: 0.0619812086224556
Epoch 60 | Loss: 0.038812484592199326
Epoch 70 | Loss: 0.024304289370775223
Epoch 80 | Loss: 0.015219198539853096
Epoch 90 | Loss: 0.009530224837362766


In [21]:
print(list(net.parameters()))

[Parameter containing:
tensor([[3.0092]], requires_grad=True), Parameter containing:
tensor([0.9492], requires_grad=True)]


In [39]:
net.forward(torch.tensor([3.0]))

tensor([9.9570], grad_fn=<AddBackward0>)

## Not MNIST

In [0]:
input_size = 784
hidden_1 = 500
num_classes = 10

num_epochs = 10
batch_size = 100
learning_rate = 1e-03

In [0]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms


train_dataset = dsets.FashionMNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)

test_dataset = dsets.FashionMNIST(root = './data', train = False, transform = transforms.ToTensor(), download = True)

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
classes = {0: 'T-shirt',
           1: 'Trouser',
           2: 'Pullover',
           3: 'Dress',
           4: 'Coat',
           5: 'Sandal',
           6: 'Shirt',
           7: 'Sneaker',
           8: 'Bag',
           9: 'Ankle Boot'}

In [0]:
class C_Net(nn.Module):
  
  def __init__(self, input_size, hidden_size, num_classes):
    super(C_Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
    
  def forward(self, X):
    return self.fc2(self.relu(self.fc1(X)))

In [0]:
cnet = C_Net(input_size, hidden_1, num_classes)

if torch.cuda.torch.cuda.is_available():
  cnet.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnet.parameters(), lr=learning_rate)

In [54]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = Variable(images.view(-1, 28*28))
    labels = Variable(labels)
    
    if torch.cuda.torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()
      
    optimizer.zero_grad()
    outputs = cnet(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if not i % 100:
      print('Epoch [%d/%d], step [%d/%d], loss: %.4f' % (epoch+1, num_epochs, 
                                                        i + 1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/10], step [1/600], loss: 2.3208
Epoch [1/10], step [101/600], loss: 2.2620
Epoch [1/10], step [201/600], loss: 2.1782
Epoch [1/10], step [301/600], loss: 2.1293
Epoch [1/10], step [401/600], loss: 2.0869
Epoch [1/10], step [501/600], loss: 2.0430
Epoch [2/10], step [1/600], loss: 1.9371
Epoch [2/10], step [101/600], loss: 1.9120
Epoch [2/10], step [201/600], loss: 1.8506
Epoch [2/10], step [301/600], loss: 1.8467
Epoch [2/10], step [401/600], loss: 1.7534
Epoch [2/10], step [501/600], loss: 1.7448
Epoch [3/10], step [1/600], loss: 1.6825
Epoch [3/10], step [101/600], loss: 1.6446
Epoch [3/10], step [201/600], loss: 1.4708
Epoch [3/10], step [301/600], loss: 1.4870
Epoch [3/10], step [401/600], loss: 1.4778
Epoch [3/10], step [501/600], loss: 1.4408
Epoch [4/10], step [1/600], loss: 1.4407
Epoch [4/10], step [101/600], loss: 1.3920
Epoch [4/10], step [201/600], loss: 1.2775
Epoch [4/10], step [301/600], loss: 1.2815
Epoch [4/10], step [401/600], loss: 1.1631
Epoch [4/10], step 

In [57]:
correct = 0
total = 0
for images, labels in test_loader:
  images = Variable(images.view(-1, 28*28))
  
  if torch.cuda.torch.cuda.is_available():
    images = images.cuda()
    labels = labels.cuda()
    
  outputs = cnet(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum()
  
print('Accuracy: ', 100 * correct/total)

Accuracy:  tensor(70, device='cuda:0')
